In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_for_train.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)
        
        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=te_x[test_i],
                y_eval=te_y[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class ShinContextualModel:
    '''
    Implementation proposal of: http://milab.snu.ac.kr/pub/BigComp2018.pdf
    '''
    def __init__(self, embedding, conv_size=3, num_layers=2):
        '''Constructor.
        # Parameters:
        embedding: numpy array representing the embedding.
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        num_layers: Number of recurrent convolutions.
        '''
        self._embedding = embedding
        self._conv_size = conv_size
        self._num_layers = num_layers

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size, self._num_layers, self._embedding_tf)
        self._pooling_tf = self._create_maxpooling_layer(self._convolution_tf)

        return self._pooling_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('pool:', str(self._pooling_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self, conv_size, num_layers, embedding):
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, filter_width]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[filter_width]))

        z = embedding
        for _ in range(num_layers):
            conv = tf.nn.conv1d(
                value=z,
                filters=W,
                stride=1,
                padding='SAME')
            bias = tf.nn.bias_add(conv, b)
            c = tf.nn.relu(bias)

            d = tf.nn.dropout(c, 0.75)
            # Add BatchNormalization or LocalResponseNormalization
            e = tf.expand_dims(d, 1)

            z = tf.nn.local_response_normalization(
                e,
                depth_radius=5,
                bias=1,
                alpha=0.001,
                beta=0.75
            )
            z = tf.squeeze(z, 1)
        # endfor
        return z

    def _create_maxpooling_layer(self, convolution):
        conv_size = convolution.shape[1].value
        embedding_size = convolution.shape[2].value

        convolution = tf.expand_dims(convolution, -1)
        pooled = tf.nn.max_pool(
            value=convolution,
            ksize=[1, conv_size, 1, 1],
            strides=[1, 1, 1, 1],
            padding='VALID')

        flat = tf.reshape(pooled, [-1, embedding_size])
        return flat

In [8]:
word_vector = embedding.astype('float32')
model = ShinContextualModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 41, 100)
conv: (?, 41, 100)
pool: (?, 100)
output: (?, 10)


In [9]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data, test_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, test_data, split_type))

random
Fold:  1
Epoch: 1, Train Loss:  1.897, Train Acc: 34.10%, Val Loss:  1.217, Val Acc: 58.40%, Time: 8.75s *
Epoch: 2, Train Loss: 0.9681, Train Acc: 68.54%, Val Loss: 0.7953, Val Acc: 73.93%, Time: 3.49s *
Epoch: 3, Train Loss: 0.6623, Train Acc: 79.02%, Val Loss:  0.645, Val Acc: 80.27%, Time: 3.46s *
Epoch: 4, Train Loss:  0.483, Train Acc: 85.17%, Val Loss: 0.5596, Val Acc: 83.40%, Time: 3.45s *
Epoch: 5, Train Loss: 0.3676, Train Acc: 88.97%, Val Loss: 0.5172, Val Acc: 83.97%, Time: 3.49s *
Epoch: 6, Train Loss: 0.2949, Train Acc: 90.99%, Val Loss: 0.4453, Val Acc: 87.25%, Time: 3.44s *
Epoch: 7, Train Loss: 0.2319, Train Acc: 93.13%, Val Loss: 0.4727, Val Acc: 86.11%, Time: 3.50s 
Epoch: 8, Train Loss: 0.1902, Train Acc: 94.10%, Val Loss: 0.4368, Val Acc: 87.18%, Time: 3.48s 
Epoch: 9, Train Loss: 0.1559, Train Acc: 94.89%, Val Loss: 0.4342, Val Acc: 88.39%, Time: 3.44s *
Epoch: 10, Train Loss: 0.1321, Train Acc: 95.81%, Val Loss: 0.4713, Val Acc: 87.82%, Time: 3.43s 
Epoch:

Epoch: 1, Train Loss:  1.892, Train Acc: 34.75%, Val Loss:  1.308, Val Acc: 56.27%, Time: 3.59s *
Epoch: 2, Train Loss: 0.9774, Train Acc: 68.70%, Val Loss: 0.8996, Val Acc: 72.58%, Time: 3.44s *
Epoch: 3, Train Loss: 0.6559, Train Acc: 79.41%, Val Loss: 0.6761, Val Acc: 79.77%, Time: 3.45s *
Epoch: 4, Train Loss: 0.4816, Train Acc: 85.18%, Val Loss:  0.562, Val Acc: 82.55%, Time: 3.44s *
Epoch: 5, Train Loss: 0.3624, Train Acc: 88.86%, Val Loss: 0.5056, Val Acc: 84.19%, Time: 3.50s *
Epoch: 6, Train Loss: 0.2824, Train Acc: 91.32%, Val Loss: 0.4714, Val Acc: 85.90%, Time: 3.46s *
Epoch: 7, Train Loss: 0.2249, Train Acc: 93.09%, Val Loss: 0.4526, Val Acc: 86.47%, Time: 3.43s *
Epoch: 8, Train Loss: 0.1823, Train Acc: 94.50%, Val Loss: 0.4684, Val Acc: 87.32%, Time: 3.45s *
Epoch: 9, Train Loss: 0.1491, Train Acc: 95.59%, Val Loss: 0.4842, Val Acc: 86.75%, Time: 3.44s 
Epoch: 10, Train Loss: 0.1185, Train Acc: 96.34%, Val Loss: 0.4901, Val Acc: 85.47%, Time: 3.46s 
Epoch: 11, Train Loss

Epoch: 1, Train Loss:   1.96, Train Acc: 32.80%, Val Loss:  1.285, Val Acc: 56.52%, Time: 3.56s *
Epoch: 2, Train Loss:  1.012, Train Acc: 67.48%, Val Loss: 0.8091, Val Acc: 74.20%, Time: 3.43s *
Epoch: 3, Train Loss: 0.6644, Train Acc: 79.49%, Val Loss: 0.6393, Val Acc: 80.68%, Time: 3.46s *
Epoch: 4, Train Loss: 0.4906, Train Acc: 84.92%, Val Loss: 0.5272, Val Acc: 84.46%, Time: 3.44s *
Epoch: 5, Train Loss:  0.375, Train Acc: 88.30%, Val Loss: 0.4717, Val Acc: 84.89%, Time: 3.44s *
Epoch: 6, Train Loss: 0.2892, Train Acc: 90.96%, Val Loss: 0.4309, Val Acc: 87.17%, Time: 3.44s *
Epoch: 7, Train Loss: 0.2275, Train Acc: 92.92%, Val Loss: 0.4293, Val Acc: 87.60%, Time: 3.44s *
Epoch: 8, Train Loss: 0.1878, Train Acc: 94.38%, Val Loss:   0.43, Val Acc: 87.60%, Time: 3.43s 
Epoch: 9, Train Loss: 0.1611, Train Acc: 94.77%, Val Loss: 0.4022, Val Acc: 88.17%, Time: 3.44s *
Epoch: 10, Train Loss: 0.1255, Train Acc: 95.86%, Val Loss: 0.4423, Val Acc: 87.88%, Time: 3.49s 
Epoch: 11, Train Loss

Epoch: 1, Train Loss:  1.914, Train Acc: 33.29%, Val Loss:   1.24, Val Acc: 59.44%, Time: 3.57s *
Epoch: 2, Train Loss:  1.012, Train Acc: 67.32%, Val Loss: 0.8385, Val Acc: 72.34%, Time: 3.46s *
Epoch: 3, Train Loss: 0.6949, Train Acc: 78.22%, Val Loss: 0.7173, Val Acc: 78.97%, Time: 3.45s *
Epoch: 4, Train Loss: 0.5036, Train Acc: 84.34%, Val Loss: 0.5899, Val Acc: 82.39%, Time: 3.46s *
Epoch: 5, Train Loss: 0.3833, Train Acc: 88.26%, Val Loss:  0.582, Val Acc: 83.61%, Time: 3.44s *
Epoch: 6, Train Loss: 0.2928, Train Acc: 91.08%, Val Loss: 0.5607, Val Acc: 83.32%, Time: 3.45s 
Epoch: 7, Train Loss:  0.232, Train Acc: 93.04%, Val Loss: 0.4927, Val Acc: 86.10%, Time: 3.45s *
Epoch: 8, Train Loss: 0.1917, Train Acc: 94.34%, Val Loss: 0.4798, Val Acc: 87.24%, Time: 3.44s *
Epoch: 9, Train Loss: 0.1546, Train Acc: 95.17%, Val Loss: 0.4835, Val Acc: 87.74%, Time: 3.44s *
Epoch: 10, Train Loss: 0.1336, Train Acc: 95.70%, Val Loss: 0.4966, Val Acc: 87.88%, Time: 3.44s *
Epoch: 11, Train Los

Epoch: 21, Train Loss: 0.03819, Train Acc: 98.75%, Val Loss:  2.315, Val Acc: 71.77%, Time: 3.49s *
Epoch: 22, Train Loss: 0.03616, Train Acc: 98.82%, Val Loss:  2.682, Val Acc: 70.58%, Time: 3.50s 
Epoch: 23, Train Loss: 0.03386, Train Acc: 98.82%, Val Loss:  2.717, Val Acc: 69.39%, Time: 3.51s 
Epoch: 24, Train Loss: 0.03148, Train Acc: 98.96%, Val Loss:  2.896, Val Acc: 69.87%, Time: 3.50s 
Epoch: 25, Train Loss: 0.0305, Train Acc: 98.95%, Val Loss:  2.661, Val Acc: 70.82%, Time: 3.50s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.47      0.79      0.59       132
                   Filter       0.68      0.77      0.72       132
    Compute Derived Value       0.84      0.32      0.46       134
            Find Extremum       0.80      0.85      0.82       127
                     Sort       0.89      0.92      0.90       119
          Determin

...

Epoch: 1, Train Loss:  2.328, Train Acc: 18.29%, Val Loss:   1.98, Val Acc: 30.86%, Time: 3.58s *
Epoch: 2, Train Loss:  1.564, Train Acc: 48.00%, Val Loss:  1.457, Val Acc: 50.24%, Time: 3.58s *
Epoch: 3, Train Loss:  1.102, Train Acc: 64.04%, Val Loss:  1.257, Val Acc: 60.53%, Time: 3.45s *
Epoch: 4, Train Loss:  0.863, Train Acc: 72.20%, Val Loss:  1.209, Val Acc: 62.84%, Time: 3.46s *
Epoch: 5, Train Loss: 0.7029, Train Acc: 77.63%, Val Loss:  1.198, Val Acc: 65.22%, Time: 3.45s *
Epoch: 6, Train Loss: 0.5666, Train Acc: 82.18%, Val Loss:  1.246, Val Acc: 64.80%, Time: 3.46s 
Epoch: 7, Train Loss: 0.4827, Train Acc: 85.18%, Val Loss:  1.282, Val Acc: 65.92%, Time: 3.44s *
Epoch: 8, Train Loss: 0.3951, Train Acc: 88.04%, Val Loss:  1.327, Val Acc: 66.48%, Time: 3.45s *
Epoch: 9, Train Loss: 0.3261, Train Acc: 90.40%, Val Loss:  1.297, Val Acc: 69.14%, Time: 3.44s *
Epoch: 10, Train Loss:  0.271, Train Acc: 91.91%, Val Loss:  1.444, Val Acc: 67.60%, Time: 3.43s 
Epoch: 11, Train Loss

Epoch: 1, Train Loss:  2.407, Train Acc: 16.77%, Val Loss:  1.955, Val Acc: 37.19%, Time: 3.60s *
Epoch: 2, Train Loss:  1.608, Train Acc: 45.69%, Val Loss:  1.425, Val Acc: 55.56%, Time: 3.47s *
Epoch: 3, Train Loss:  1.109, Train Acc: 63.95%, Val Loss:  1.215, Val Acc: 65.33%, Time: 3.47s *
Epoch: 4, Train Loss: 0.8552, Train Acc: 72.77%, Val Loss:  1.165, Val Acc: 66.76%, Time: 3.49s *
Epoch: 5, Train Loss: 0.6882, Train Acc: 78.76%, Val Loss:   1.19, Val Acc: 66.40%, Time: 3.47s 
Epoch: 6, Train Loss: 0.5732, Train Acc: 82.13%, Val Loss:  1.189, Val Acc: 68.27%, Time: 3.46s *
Epoch: 7, Train Loss: 0.4657, Train Acc: 85.67%, Val Loss:   1.26, Val Acc: 66.19%, Time: 3.47s 
Epoch: 8, Train Loss: 0.3934, Train Acc: 87.86%, Val Loss:  1.247, Val Acc: 68.27%, Time: 3.59s 
Epoch: 9, Train Loss:  0.332, Train Acc: 90.08%, Val Loss:  1.289, Val Acc: 68.77%, Time: 3.45s *
Epoch: 10, Train Loss: 0.2656, Train Acc: 92.17%, Val Loss:  1.353, Val Acc: 68.34%, Time: 3.46s 
Epoch: 11, Train Loss: 

Epoch: 1, Train Loss:  2.026, Train Acc: 29.86%, Val Loss:  1.641, Val Acc: 45.21%, Time: 3.51s *
Epoch: 2, Train Loss:   1.12, Train Acc: 64.00%, Val Loss:  1.334, Val Acc: 58.73%, Time: 3.39s *
Epoch: 3, Train Loss: 0.7638, Train Acc: 76.06%, Val Loss:  1.266, Val Acc: 62.97%, Time: 3.40s *
Epoch: 4, Train Loss: 0.5779, Train Acc: 81.93%, Val Loss:  1.249, Val Acc: 65.03%, Time: 3.41s *
Epoch: 5, Train Loss: 0.4425, Train Acc: 86.46%, Val Loss:  1.258, Val Acc: 66.61%, Time: 3.37s *
Epoch: 6, Train Loss: 0.3417, Train Acc: 89.34%, Val Loss:  1.277, Val Acc: 66.97%, Time: 3.36s *
Epoch: 7, Train Loss:  0.273, Train Acc: 91.83%, Val Loss:  1.339, Val Acc: 67.09%, Time: 3.37s *
Epoch: 8, Train Loss: 0.2252, Train Acc: 92.90%, Val Loss:  1.525, Val Acc: 66.36%, Time: 3.39s 
Epoch: 9, Train Loss: 0.1845, Train Acc: 94.33%, Val Loss:  1.557, Val Acc: 67.33%, Time: 3.37s *
Epoch: 10, Train Loss: 0.1512, Train Acc: 95.31%, Val Loss:  1.676, Val Acc: 65.33%, Time: 3.37s 
Epoch: 11, Train Loss

Epoch: 4, Train Loss: 0.5325, Train Acc: 83.52%, Val Loss:   1.31, Val Acc: 62.92%, Time: 3.38s *
Epoch: 5, Train Loss: 0.4008, Train Acc: 87.67%, Val Loss:  1.357, Val Acc: 62.63%, Time: 3.39s 
Epoch: 6, Train Loss: 0.3155, Train Acc: 90.45%, Val Loss:    1.5, Val Acc: 61.99%, Time: 3.39s 
Epoch: 7, Train Loss: 0.2457, Train Acc: 92.45%, Val Loss:  1.697, Val Acc: 60.41%, Time: 3.37s 
Epoch: 8, Train Loss: 0.1984, Train Acc: 94.10%, Val Loss:   1.84, Val Acc: 59.47%, Time: 3.45s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.62      0.39      0.48       167
                   Filter       0.63      0.31      0.41       150
    Compute Derived Value       0.74      0.28      0.41       213
            Find Extremum       0.60      0.74      0.66       168
                     Sort       0.80      0.77      0.78       146
          Determine Range  

Epoch: 8, Train Loss:  0.202, Train Acc: 93.95%, Val Loss:  1.764, Val Acc: 64.48%, Time: 3.40s 
Epoch: 9, Train Loss: 0.1673, Train Acc: 94.99%, Val Loss:   1.81, Val Acc: 64.55%, Time: 3.42s 
Epoch: 10, Train Loss: 0.1383, Train Acc: 95.78%, Val Loss:  1.931, Val Acc: 62.28%, Time: 3.41s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.74      0.54      0.63       206
                   Filter       0.44      0.44      0.44       202
    Compute Derived Value       0.75      0.31      0.44       198
            Find Extremum       0.45      0.84      0.59       189
                     Sort       0.59      0.88      0.71        84
          Determine Range       0.34      0.38      0.36       109
Characterize Distribution       0.80      0.80      0.80       137
           Find Anomalies       0.72      0.63      0.67       186
                  Cl